# SiFThon - سيفثون

In [ ]:
# @title إعداد المتغيرات
import ipywidgets as widgets
from IPython.display import display

session_code_input = widgets.Text(
    description="كود الجلسة:",
    placeholder="أدخل كود الجلسة هنا..."
)
api_id_input = widgets.Text(
    description="API ID:",
    placeholder="أدخل الـ API ID هنا..."
)
api_hash_input = widgets.Text(
    description="API Hash:",
    placeholder="أدخل الـ API Hash هنا..."
)
bot_token_input = widgets.Text(
    description="توكن البوت:",
    placeholder="أدخل توكن البوت هنا..."
)

print("=" * 50)
print("إعداد متغيرات السورس")
print("=" * 50)

display(session_code_input)
display(api_id_input)
display(api_hash_input)
display(bot_token_input)

output = widgets.Output()
display(output)

def validate_inputs():
    session = session_code_input.value.strip()
    api_id = api_id_input.value.strip()
    api_hash = api_hash_input.value.strip()
    bot_token = bot_token_input.value.strip()
    
    with output:
        output.clear_output()
        if not all([session, api_id, api_hash, bot_token]):
            print("خطأ: يرجى ملء جميع الحقول المطلوبة")
            return False
        if not api_id.isnumeric():
            print("خطأ: الـ API ID يجب أن يحتوي على أرقام فقط")
            return False
        return True

def confirm_input(b):
    if validate_inputs():
        session_code_input.disabled = True
        api_id_input.disabled = True
        api_hash_input.disabled = True
        bot_token_input.disabled = True
        confirm_button.disabled = True
        with output:
            print("=" * 40)
            print("تم تأكيد المدخلات بنجاح")
            print("يمكنك الآن متابعة التنصيب")
            print("=" * 40)

confirm_button = widgets.Button(
    description="تأكيد البيانات",
    button_style='success',
    tooltip='انقر لتأكيد صحة البيانات المدخلة'
)
confirm_button.on_click(confirm_input)
display(confirm_button)

In [ ]:
# @title تشغيل سورس سيفثون
import os
import requests
import zipfile
import subprocess

print("=" * 50)
print("بدء تشغيل سورس سيفثون")
print("=" * 50)

try:
    API_ID = api_id_input.value
    API_HASH = api_hash_input.value
    SESSION = session_code_input.value
    BOT_TOKEN = bot_token_input.value
except:
    print("-" * 40)
    print("لم يتم تعيين المتغيرات")
    print("يرجى تعيين المتغيرات في القسم السابق")
    print("-" * 40)
    exit()

print("⏳ جاري تحميل السورس...")
url = "http://especi1.serv00.net/SiFThon.zip"
response = requests.get(url)
with open("SiFThon.zip", "wb") as f:
    f.write(response.content)

print("⏳ جاري فك الضغط...")
with zipfile.ZipFile("SiFThon.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

# تأكد من وجود المجلد قبل إنشاء .env
if not os.path.exists("SiFThon"):
    os.makedirs("SiFThon")

print("⏳ جاري تثبيت المتطلبات...")
os.system("pip install -q pyrogram telethon requests Pillow python-decouple yt-dlp")

print("⏳ جاري إنشاء ملف الإعدادات...")
env_content = f"""API_ID={API_ID}
API_HASH={API_HASH}
SESSION={SESSION}
BOT_TOKEN={BOT_TOKEN}
"""
with open('SiFThon/.env', 'w') as file:
    file.write(env_content)

print("=" * 50)
print("🚀 بدء تشغيل السورس")
print("=" * 50)

# ✅ تشغيل الملف الرئيسي مباشرة بدل -m
main_file = None
for f in os.listdir("SiFThon"):
    if f.endswith('.py') and 'sython' in f.lower():
        main_file = f
        break

if main_file:
    process = subprocess.Popen(['python', f'SiFThon/{main_file}'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    try:
        for line in process.stdout:
            print(line, end='')
    except KeyboardInterrupt:
        process.terminate()
        print('\n❌ تم إيقاف التشغيل يدويًا')
else:
    print('❌ لم يتم العثور على الملف الرئيسي لتشغيل السورس')